## Demonstration of loading data, using no resolver

#### Add the path of modules of the project to sys

In [5]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

#### Import necessary libraries

In [6]:
import pathlib

from pepys_import.core.formats.repl_file import REPFile
from pepys_import.core.store.data_store import DataStore
from pepys_import.resolvers.command_line_resolver import CommandLineResolver
from pepys_import.core.store.table_summary import TableSummary, TableSummarySet
from pepys_import.utils.unit_utils import convert_speed, convert_heading

#### Declaring global variables

In [7]:
RESOURCES_PATH = os.path.join('..', 'repl_files')

data_store = DataStore("", "", "", 0, ":memory:", db_type='sqlite',
                      missing_data_resolver=CommandLineResolver())
data_store.initialise()
file_path = os.path.join(RESOURCES_PATH, "missing_platform.rep")
rep = REPFile(str(file_path))

______                      _                            _   
| ___ \                    (_)                          | |  
| |_/ /__ _ __  _   _ ___   _ _ __ ___  _ __   ___  _ __| |_ 
|  __/ _ \ '_ \| | | / __| | | '_ ` _ \| '_ \ / _ \| '__| __|
| | |  __/ |_) | |_| \__ \ | | | | | | | |_) | (_) | |  | |_ 
\_|  \___| .__/ \__, |___/ |_|_| |_| |_| .__/ \___/|_|   \__|
         | |     __/ | ______          | |                   
         |_|    |___/ |______|         |_|                   

Software Version :  0.0.4 


Database Type :  sqlite
Database Name :  :memory:
Database Host :  
-------------------------------------------------------------


In [8]:
with data_store.session_scope():
    privacy = data_store.add_to_privacies("PRIVACY-1")
    datafile = data_store.get_datafile(rep.filepath, rep.datafile_type)
    for line_number, rep_line in enumerate(rep.lines, 1):
        platform = data_store.get_platform(rep_line.get_platform(),"Fisher", "UK", "PRIVACY-1")
        sensor = platform.get_sensor(data_store=data_store, sensor_name="GPS")
        state = datafile.create_state(sensor, rep_line.timestamp)
        speed = convert_speed(rep_line.speed, line_number)
        state.speed = speed
        heading = convert_heading(rep_line.heading, line_number)
        state.heading = heading
        state.location = rep_line.get_location()
        state.privacy = privacy.privacy_id
        
    if datafile.validate():
        datafile.commit(data_store.session)
        
    states_sum = TableSummary(data_store.session, data_store.db_classes.State)
    platforms_sum = TableSummary(
        data_store.session, data_store.db_classes.Platform
    )
    first_table_summary_set = TableSummarySet([states_sum, platforms_sum])
    print(first_table_summary_set.report())

UnsupportedOperation: fileno

In [9]:
from prompt_toolkit import prompt
prompt("test")

UnsupportedOperation: fileno